<a href="https://colab.research.google.com/github/padthamapornmei/Tool-Science/blob/main/Time_Series_SARIMA_Tools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pmdarima

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from pmdarima import auto_arima # np.__version__ could be 1.26.4
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.statespace.sarimax import SARIMAX
import warnings

%matplotlib inline

In [ ]:
import pandas as pd # Importing the pandas library and assigning it the alias 'pd'

data_path = '/content/avocado.csv'
df = pd.read_csv(data_path)

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
df = df.drop('Unnamed: 0', axis = 1)

df.Date = pd.to_datetime(df.Date, format='%Y-%m-%d')

In [ ]:
df1 = df[df.type=='conventional'].groupby(['region', 'Date']).agg({'AveragePrice': 'mean', 'Total Volume': 'mean', '4046': 'mean', '4225': 'mean', '4770': 'mean', 'Total Bags': 'mean', 'Small Bags': 'mean', 'Large Bags': 'mean', 'XLarge Bags': 'mean'}).reset_index()
df2 = df[df.type=='organic'].groupby(['region', 'Date']).agg({'AveragePrice': 'mean', 'Total Volume': 'mean', '4046': 'mean', '4225': 'mean', '4770': 'mean', 'Total Bags': 'mean', 'Small Bags': 'mean', 'Large Bags': 'mean', 'XLarge Bags': 'mean'}).reset_index()

In [ ]:
df1['year'] = df1.Date.dt.year
df1['month'] = df1.Date.dt.month
df1['day'] = df1.Date.dt.day

df2['year'] = df2.Date.dt.year
df2['month'] = df2.Date.dt.month
df2['day'] = df2.Date.dt.day

In [ ]:
top_5_avg_price = df.groupby('region')['AveragePrice'].mean().reset_index()
top_5_avg_price = top_5_avg_price.sort_values('AveragePrice', ascending=False)

In [ ]:
cheapest_avg_price = top_5_avg_price.tail(1).reset_index()
avg_US_price = top_5_avg_price[top_5_avg_price.region=='TotalUS'][['region', 'AveragePrice']]
top_5_avg_price = top_5_avg_price.head()

In [ ]:
combined_avg_price = pd.concat([top_5_avg_price, avg_US_price, cheapest_avg_price])
combined_avg_price = combined_avg_price.reset_index(drop=True)
combined_avg_price = combined_avg_price.drop('index', axis=1)

In [ ]:
combined_avg_price

,region,AveragePrice
0,HartfordSpringfield,1.818639
1,SanFrancisco,1.804201
2,NewYork,1.727574
3,Philadelphia,1.632130
4,Sacramento,1.621568
5,TotalUS,1.319024
6,Houston,1.047929


In [ ]:
import plotly.express as px # Import the plotly.express library and assign it the alias 'px'

fig = px.bar(combined_avg_price, x= combined_avg_price['region'], y=combined_avg_price['AveragePrice'], title='Regions by average price')
fig.update_layout(xaxis_title='Regions', yaxis_title='Average price')
fig.show()

In [ ]:
fig = px.bar(combined_avg_price, x= combined_avg_price['region'], y=(combined_avg_price['AveragePrice'] / combined_avg_price['AveragePrice'].max()) * 100, title='Regions by average price in percentage')
fig.update_layout(xaxis_title='Regions', yaxis_title='Percentage (%)')
fig.show()

In [ ]:
import plotly.graph_objects as go # Import the plotly.graph_objects library and assign it the alias 'go'

fig = go.Figure()

for i in combined_avg_price.region.unique():
    fig.add_trace(go.Scatter(
            x=df1[df1.region==i]['Date'],
            y=df1[df1.region==i]['AveragePrice'],
            mode='lines',
            hovertemplate="Price: %{y:.2f}",
            name=i
        ))

fig.update_layout(
    width=1100,
    height=600,
    title='Average price by Region',
    xaxis_title='Date',
    yaxis_title='Average Price',
    hovermode="x",
    xaxis_tickformat='%m/%Y',
    xaxis_showgrid=False
)

fig.show()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=df1[df1.region == 'TotalUS']['Date'],
                         y=df1[df1.region == 'TotalUS']['AveragePrice'],
                         mode='lines',
                         name='conventional',
                         line=dict(color='#00ffff')))

fig.add_trace(go.Scatter(x=df2[df2.region == 'TotalUS']['Date'],
                         y=df2[df2.region == 'TotalUS']['AveragePrice'],
                         mode='lines',
                         name='organic',
                         line=dict(color='#ffadff')))

fig.update_layout(width=1100,
                  height=600,
                  title='Average price convertional and organic avocados in Total US',
                  plot_bgcolor='#ccf2ff',
                  hovermode="x",
                  paper_bgcolor='white',
                  xaxis_title='Date',
                  yaxis_title='Average Price',
                  xaxis_showgrid=False
)

fig.show()

In [ ]:
df2.loc[df2['AveragePrice'] == 1, 'AveragePrice'] = df2.loc[df2['year'] == 2015, 'AveragePrice'].mean()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=df1[df1.region == 'TotalUS']['Date'],
                         y=df1[df1.region == 'TotalUS']['AveragePrice'],
                         mode='lines',
                         name='conventional',
                         line=dict(color='#00ffff')))

fig.add_trace(go.Scatter(x=df2[df2.region == 'TotalUS']['Date'],
                         y=df2[df2.region == 'TotalUS']['AveragePrice'],
                         mode='lines',
                         name='organic',
                         line=dict(color='#ffadff')))

fig.update_layout(width=1100,
                  height=600,
                  title='Average price convertional and organic avocados in Total US',
                  plot_bgcolor='#ccf2ff',
                  hovermode="x",
                  paper_bgcolor='white',
                  xaxis_title='Date',
                  yaxis_title='Average Price',
                  xaxis_showgrid=False
)

fig.show()


In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=df1[df1.region == 'TotalUS']['Date'],
                         y=df2[df2.region == 'TotalUS']['AveragePrice']/df1[df1.region == 'TotalUS']['AveragePrice']*100,
                         mode='lines',
                         name='organic/covertional',
                         line=dict(color='#ffadff'),
                         hovertemplate="%{y:.2f}%",))

fig.update_layout(width=1100,
                  height=600,
                  title='How much more expensive are organic avocados than convertional',
                  plot_bgcolor='#ccf2ff',
                  hovermode="x",
                  paper_bgcolor='white',
                  xaxis_title='Date',
                  yaxis_title='Percent',
                  xaxis_showgrid=False
)

fig.show()


In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=df1[df1.region == 'TotalUS']['Date'],
                         y=df1[df1.region == 'TotalUS']['Total Volume'],
                         mode='lines',
                         name='conventional',
                         line=dict(color='#00ffff')))

fig.add_trace(go.Scatter(x=df2[df2.region == 'TotalUS']['Date'],
                         y=df2[df2.region == 'TotalUS']['Total Volume'],
                         mode='lines',
                         name='organic',
                         line=dict(color='#ffadff')))

fig.update_layout(width=1100,
                  height=600,
                  title='Total US volume organic and convertional avocados',
                  plot_bgcolor='#ccf2ff',
                  hovermode="x",
                  paper_bgcolor='white',
                  xaxis_title='Date',
                  yaxis_title='Volume',
                  xaxis_showgrid=False
)

fig.update_xaxes(showspikes=True, spikecolor="gray", spikesnap="cursor", spikemode="across", spikethickness=1.5)
fig.update_layout(spikedistance=1000, hoverdistance=100)

fig.show()

In [ ]:
df1['Income'] = df1['Total Volume']*df1['AveragePrice']
df2['Income'] = df2['Total Volume']*df2['AveragePrice']

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=df1[df1.region == 'TotalUS']['Date'],
                         y=df1[df1.region == 'TotalUS']['Income'],
                         mode='lines',
                         name='conventional',
                         line=dict(color='#00ffff')))

fig.add_trace(go.Scatter(x=df2[df2.region == 'TotalUS']['Date'],
                         y=df2[df2.region == 'TotalUS']['Income'],
                         mode='lines',
                         name='organic',
                         line=dict(color='#ffadff')))

fig.update_layout(width=1100,
                  height=600,
                  title='Total sales from convertional avocados and organic',
                  plot_bgcolor='#ccf2ff',
                  hovermode="x",
                  paper_bgcolor='white',
                  xaxis_title='Date',
                  yaxis_title='Income',
                  xaxis_showgrid=False
)

fig.update_xaxes(showspikes=True, spikecolor="gray", spikesnap="cursor", spikemode="across", spikethickness=1.5)
fig.update_layout(spikedistance=1000, hoverdistance=100)

fig.show()


In [ ]:
correl_con = df1[['AveragePrice','Total Volume', 'Income']].corr()
correl_org = df2[['AveragePrice','Total Volume', 'Income']].corr()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Heatmap(z=correl_con.values, x=list(correl_con.columns), y=list(correl_con.index), colorscale='mint', texttemplate='%{z:.2f}'))
fig.update_layout(title='Convertional avocado correlation', xaxis=dict(side='top'))
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Heatmap(z=correl_org.values, x=list(correl_org.columns), y=list(correl_org.index), colorscale='mint', texttemplate='%{z:.2f}'))
fig.update_layout(title='Organic avocado correlation', xaxis=dict(side='top'))
fig.show()

In [ ]:
rd = [(n, (df1[df1.year==n]['AveragePrice'].mean()-df1[df1.year==n-1]['AveragePrice'].mean())/df1[df1.year==n]['AveragePrice'].mean()) for n in range(2016, 2019)]

years, ratios = zip(*rd)
ratios = [f"{ratio * 100:.2f}%" for ratio in ratios]

table_data = list(zip(*[years, ratios]))

fig = go.Figure(data=[go.Table(
    header=dict(fill_color='white',
                align='left'),
    cells=dict(values=[['Year','Value']]+table_data,
               fill_color='paleturquoise',
               align='left'))
])

fig.show()

In [ ]:
regression_data = df1[(df1.region == 'TotalUS') & (df1.year < 2018)][['Date', 'AveragePrice']]
regression_data = regression_data.set_index('Date')

regression_data = regression_data.resample('M').mean()

<ipython-input-27-2c7ac46177ea>:4: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.



In [ ]:
import statsmodels.api as sm # Importing the statsmodels library and assign it the alias 'sm'
from sklearn.metrics import mean_squared_error # Importing mean_squared_error from sklearn.metrics

# ... Your existing code ...
model = sm.tsa.statespace.SARIMAX(regression_data,  order = (2, 1, 2), seasonal_order =(1, 0, 0, 12)) # Calling SARIMAX class using 'sm.tsa.statespace.SARIMAX'
result = model.fit()
test_model = result.predict(0, 35)
predictions = result.predict(35, 60)

print('MSE: ',mean_squared_error(regression_data[1:], test_model[1:]))

MSE:  0.009255214782370881


In [ ]:
model_df = pd.concat([test_model[1:], regression_data[1:]])
model_df.columns = ['ModelPrice', 'SpecifiedPrice']

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=model_df.index, y=model_df[model_df['ModelPrice'].isnull()]['SpecifiedPrice'], name="Specified Price", line=dict(color='#00ffff')))
fig.add_trace(go.Scatter(x=model_df.index, y=model_df[model_df['SpecifiedPrice'].isnull()]['ModelPrice'], name="Model price", line=dict(color='#ffadff')))

fig.update_layout(title="Check model price", yaxis_title="Price", width=1100, height=600, plot_bgcolor='#ccf2ff', hovermode="x", paper_bgcolor='white', xaxis_showgrid=False)

fig.show()

In [ ]:
prediction_df = pd.concat([predictions, regression_data[1:]])
prediction_df.columns = ['PredictionPrice', 'SpecifiedPrice']

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=prediction_df[prediction_df['PredictionPrice'].isnull()]['SpecifiedPrice'].index, y=prediction_df[prediction_df['PredictionPrice'].isnull()]['SpecifiedPrice'], name="Specified Price", line=dict(color='#00ffff')))
fig.add_trace(go.Scatter(x=prediction_df.index, y=prediction_df[prediction_df['SpecifiedPrice'].isnull()]['PredictionPrice'], name="Prediction Price", line=dict(color='#ffadff')))

fig.update_layout(title="Prediction price up to 2020", yaxis_title="Price", width=1100, height=600, plot_bgcolor='#ccf2ff', hovermode="x", paper_bgcolor='white', xaxis_showgrid=False)

fig.show()

In [ ]:
import statsmodels.api as sm # Import the statsmodels API

# Instead of directly using SARIMAX, use sm.tsa.statespace.SARIMAX
model = sm.tsa.statespace.SARIMAX(regression_data, order=(0, 1, 0), seasonal_order=(1, 1, 0, 12))
result = model.fit()
test_model = result.predict(0, 35, type='levels')
predictions = result.predict(35, 60, type='levels')

mean_squared_error(regression_data[1:], test_model[1:])

0.02013799368808129

In [ ]:
prediction_df = pd.concat([predictions, regression_data[1:]])
prediction_df.columns = ['PredictionPrice', 'SpecifiedPrice']

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=prediction_df[prediction_df['PredictionPrice'].isnull()]['SpecifiedPrice'].index, y=prediction_df[prediction_df['PredictionPrice'].isnull()]['SpecifiedPrice'], name="Specified Price", line=dict(color='#00ffff')))
fig.add_trace(go.Scatter(x=prediction_df.index, y=prediction_df[prediction_df['SpecifiedPrice'].isnull()]['PredictionPrice'], name="Prediction Price", line=dict(color='#ffadff')))

fig.update_layout(title="Prediction price up to 2020", yaxis_title="Price", width=1100, height=600, plot_bgcolor='#ccf2ff', hovermode="x", paper_bgcolor='white', xaxis_showgrid=False)

fig.show()

In [ ]:
import numpy as np
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import r2_score

# 1. เตรียมข้อมูล
df_grouped = df.groupby("Date")["Total Volume"].sum()

# แบ่ง Train (80%) และ Test (20%)
train_size = int(len(df_grouped) * 0.8)
train, test = df_grouped[:train_size], df_grouped[train_size:]

# 2. สร้างและ Train โมเดล SARIMA
model = SARIMAX(train, order=(1,1,1), seasonal_order=(1,1,1,52))
sarima_fit = model.fit()

# 3. พยากรณ์ยอดขายในช่วง test set
forecast = sarima_fit.forecast(steps=len(test))

# 4. คำนวณค่า R^2
r2 = r2_score(test, forecast)
print(f"R^2 Score: {r2:.4f}")


R^2 Score: 0.6975
